### Collaborative Filtering
* 유저가 좋아하는 것을 다른 유저와 유사성에 기반해 예측하는 것.
* Advantages: 아이템 자체에 대한 이해가 없이, 영화와 같은 복잡한 항목을 정확하게 추천 할 수 있음. 
* 보통 추천 시스템에는 user similarity 와 item similarity가 있음.
* Task 1: finding similar animes
* Task 2: finding similar users
* Task 3: Recommending Animes for a random user

In [ ]:
INPUT_DIR = "../input/anime-recommendation-database-2020"
!ls {INPUT_DIR}

In [ ]:
import numpy as np
import pandas as pd

rating_df = pd.read_csv(INPUT_DIR + "/animelist.csv",
                       low_memory = False,
                        usecols=["user_id","anime_id","rating"]
                       )
rating_df.head()

In [ ]:
# User should rate at least 400
n_ratings = rating_df['user_id'].value_counts() # 칼럼의 unique값을 카운트. 왼쪽이 user_id, rating 개수.
rating_df = rating_df[rating_df['user_id'].isin(n_ratings[n_ratings>=400])].copy() #isin 은 해당 리스트를 포함하고 있는 행을 골라줌.
len(rating_df)

In [ ]:
# 0, 1.0 사이로 스케일링
min_rating = min(rating_df['rating'])
max_rating = max(rating_df['rating'])
rating_df['rating'] = rating_df['rating'].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values.astype(np.float64)

AvgRating = np.mean(rating_df['rating'])
print("Avg", AvgRating)

In [ ]:
# Removing Duplicated Rows
duplicates = rating_df.duplicated()

if duplicates.sum() > 0:
    print("> {} duplicates".format(duplicates.sum()))
    rating_df = rating_df[~duplicates] # ~은 not의 의미. 즉, 중복된 것이 없는 것만 뽑아낸 것이라고 보면 될 듯.

print("> {} duplicates".format(rating_df.duplicated().sum()))

In [ ]:
g = rating_df.groupby("user_id")['rating'].count() # count() : Count the number of (not NULL) values in each row 유저가 평점을 준 개수 구하기
top_users = g.dropna().sort_values(ascending=False)[:20] # dropna(): Remove missing values. 결측치 제거 후 역순으로 20개 정렬(별점이 많은 순으로)
top_r = rating_df.join(top_users, rsuffix="_r", how="inner", on="user_id") # user_id(on)가 겹치는 항목에 대해 두 프레임 합치기
top_r

g = rating_df.groupby('anime_id')['rating'].count()
top_animes = g.dropna().sort_values(ascending=False)[:20]
top_r = top_r.join(top_animes, rsuffix="_r", how = "inner", on = "anime_id")

pd.crosstab(top_r.user_id, top_r.anime_id, top_r.rating, aggfunc = np.sum)

### Data Preprocessing

In [ ]:
# Encoding categorical data
# 유니크한 user_id 도출 -> ex. 400:1, 402:4로 user 테이블에 매핑시켜 둠 -> categorical한 데이터를 인코딩?
user_ids = rating_df['user_id'].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
user_encoded2user = {i: x for i, x in enumerate(user_ids)}
rating_df['user'] = rating_df['user_id'].map(user2user_encoded) # user2user_encoded에 있는 값으로 매핑시킴
n_users = len(user2user_encoded)

anime_ids = rating_df['anime_id'].unique().tolist()
anime2anime_encoded = {x: i for i, x in enumerate(anime_ids)}
anime_encoded2anime = {i: x for i, x in enumerate(anime_ids)}
rating_df['anime'] = rating_df['anime_id'].map(anime2anime_encoded)
n_animes = len(anime2anime_encoded)

print("Num of users: {}, Num of animes: {}".format(n_users, n_animes))
print("Min rating: {}, Max rating: {}".format(min(rating_df['rating']), max(rating_df['rating'])))

In [ ]:
# Shuffle
# frac: 추출할 데이터 비율. 비복원의 경우 frac = 0 ~ 1 사이의 값
# n: 추출할 데이터의 개수.
rating_df = rating_df.sample(frac=1, random_state=73)

X = rating_df[['user', 'anime']].values
y = rating_df['rating']

In [ ]:
# Split
test_set_size = 10000
train_indices = rating_df.shape[0] - test_set_size

X_train, X_test, y_train, y_test = (
    X[:train_indices],
    X[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

print("> Train set ratings: {}".format(len(y_train)))
print("> Test set ratings: {}".format(len(y_test)))

In [ ]:
X_train_array = [X_train[:,0], X_train[:,1]]
X_test_array = [X_test[:,0], X_test[:,1]]

In [ ]:
# Accelerator check
import tensorflow as tf

TPU_INIT = True

if TPU_INIT:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    !nvidia-smi

print(tf.__version__)

### Model Building

In [ ]:
!pip install keras

In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-gpu

In [ ]:
from tensorflow import keras 
from keras import layers
import tensorflow as tf
from keras.models import Model
from keras.optimizers import Adam

In [ ]:
# Embedding layers
from keras.layers import Add, Activation, Lambda, BatchNormalization, Concatenate, Dropout,
Input, Embedding, Dot, Reshape, Dense, Flatten

def RecommenderNet():
    embedding_size = 128
    
    user = Input(name = "user", shape=[1]) # Input(): keras 텐서를 인스턴스화하는 데 사용.
    user_embedding = Embedding(name = "user_embedding", 
                          input_dim = n_users,
                          output_dim = embedding_size)(user)
    anime = Input(name="anime", shape=[1])
    anime_embedding = Embedding(name = "anime_embedding",
                               input_dim = n_animes,
                               output_dim = embedding_size)(anime)
    
    # x = concatenate()([user_embedding, anime_embedding])
    x = Dot(name = "dot_product", normalize=True, axes=2)([user_embedding, anime_embedding])
    x = Flatten()(x)
    
    x = Dense(1, kernel_initializer = "he_normal")(x)     # kernerl_initializer: 가중치 초기화 함수
    x = BatchNormalization()(x)
    x = Activation("sigmoid")(x)
    
    model = Model(inputs=[user,anime], outputs =x)
    # mae: mean absolute error, mse: mean sqaured error
    model.compile(loss="binary_crossentropy", metrics=["mae",",mse"], optimizer="Adam")
    
    return model

if TPU_INIT:
    with tpu_strategy.scope():
        model = RecommenderNet()
else:
    model = RecommenderNet()
    
model.summary()
    

In [ ]:
# Callbacks
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, ReduceLROplateau

start_lr = 0.00001
min_lr = 0.00001
max_lr = 0.00005
batch_size = 10000

if TPU_INIT:
    max_lr = max_lr * tpu_strategy.num_replicas_in_sync
    batch_size = batch_size * tpu_strategy.num_replicas_in_sync
    
rampup_epochs = 5
sustain_epochs = 0
exp_decay = .8

def lrfn(epoch):
    if epoch < rampup_epochs:
        return (max_lr - start_lr)/ rampup_epochs * epoch + start_lr
    elif epoch < rampup_epochs + sustain_epochs:
        return max_lr
    else:
        return (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr
    
lr_callback = LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=0)

checkpoint_filepath = "./weights.h5"

model_checkpoints = ModelCheckpoint(filepath = checkpoint_filepath,
                                   save_weights_only = True, # False이면 모델 전체를 저장. True이면 가중치만 저장
                                    monitor = "val_loss",
                                    mode = "min",# auto/ min/ max (monitor가 val_acc이면 max, val_loss이면 min)
                                    save_best_only = True #
                                   )
early_stopping = EarlyStopping(patience = 3, monitor="val_loss",
                              mode="min", restore_best_weights =True) # False이면 훈련 마지막 단계에서 얻은 모델 가중치가 사용됨. True이면 epoch 중 최상의 값으로 가중치 복원함,
my_callbacks = [
    model_checkpoints,
    lr_callback,
    early_stopping,
]

In [ ]:
# Model training
history = model.fit(
    x = X_train_array,
    y = y_train,
    batch_size = batch_size,
    epochs = 20,
    verbose = 1, # show progress bar
    validation_data = (X_test_array, y_test),
    callbacks = my_callbacks
)

model.load_weights(checkpoint_filepath)

In [ ]:
# Training results
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(history.history["loss"][0:-2])
plt.plot(history.history["val_loss"][0:-2])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train","test"], loc = "upper left")
plt.show()

### Extracting weights from model


In [ ]:
def extract_weights(name, model):
    weight_layer = model.get_layer(name)
    weights = weight_layer.get_weights()[0]
    weights = weights / np.linalg.norm(weights, axis=1).reshape((-1,1))
    return weights

anime_weights = extract_weights("anime_embedding", model)
user_weights = extract_weights("user_embedding", model)

**anime meta data**

In [ ]:
df = pd.read_csv(INPUT_DIR + "/anime.csv", low_memory = True)
df = df.replace("Unknown", np.nan)

In [ ]:
# Fixing Names
def getAnimeName(anime_id):
    try:
        name = df[df.anime_id == anime_id].eng_version.values[0]
        if name is np.nan:
            name = df[df.anime_id == anime_id].Name.values[0]
    except:
        print("error")
    return name

df["anime_id"] = df["MAL_ID"]
df["eng_version"] = df["English name"]
df["eng_version"] = df.anime_id.apply(lambda x: getAnimeName(x))

df.sort_values(by = ["Score"],
              inplace=True, 
               ascending = False, 
               kind = "quicksort",
               na_position = "last" 
              )
df = df[["anime_id", "eng_version","Score","Genders"
         ,"Episodes","Type", "Premiered","Members"]]

In [ ]:
def getAnimeFrame(anime):
    if isinstance(anime, int):
        return df[df.anime_id == anime]
    if isinstance(anime, str):
        return df[df.eng_version == anime]

**sypnopsis data**

In [ ]:
cols = ["MAL_ID", "Name", "Genders","sypnopsis"]
sypnopsis_df = pd.read_csv(INPUT_DIR + "/anime_with_synopsis.csv", usecols=cols)

def getSypnopsis(anime):
    if isinstance(anime, int):
        return sypnopsis_df[sypnopsis_df.MAL_ID == anime].sypnopsis.values[0]
    if isinstance(anime, str):
        return sypnopsis_df[sypnopsis_df.Name == anime].sypnopsis.values[0]

## Task1: Finding Similar Animes (Item Based Rcommendation)

In [ ]:
# pd.reset_option("all")
pd.reset_option("max_colwidth", None)

def find_similar_animes(name, n = 10, return_dist = False, neg=False):
    try:
        index = getAnimeFrame(name).anime_id.values[0]
        encoded_index = anime2anime_encoded.get(index)
        weights = anime_weights
        
        dists = np.dot(weigths, weights[encoded_index])
        sorted_dists = np.argsort(dists)
        
        n += 1
        
        if neg:
            closest = sorted_dists[:n]
        else:
            closest = sorted_dists[-n:]
            
        print("animes closest to {}".format(name))
        
        if return_dist:
            return dists, closest
        
        rindex = df
        
        SimilarityArr = []
        
        for close in closest:
            decoded_id = anime_encoded2anime.get(close)
            sypnopsis = getSypnopsis(decoded_id)
            anime_frame = getAnimeFrame(decoded_id)
            
            anime_name = anime_frame.eng_version.values[0]
            genre = anime_frame.Genders.values[0]
            similarity = dists[close]
            SimilarityArr.append({"anime_id":decoded_id, "name":anime_name,
                                 "similarity": similarity, "genre":genre,
                                 "sypnopsis":sypnopsis})
        Frame = pd.DataFrame(SimilarityArr).sort_values(by="similarity", ascending = False)
        return Frame[Frame.anime_id != index].drop(['anime_id'], axis=1)
    
    except:
        print('{}!, Not Found in Anime list'.format(name))

In [ ]:
find_similar_animes("Dragon Ball Z", n=5, neg=False)

In [ ]:
find_similar_animes("Your name", n=5, neg= False)

### Task2: Finding Similar Users (User Based Recommendation)

In [ ]:
print("> picking up random user")

# 500개 미만의 평가를 한 유저 한 명을 랜덤추출
ratings_per_user = rating_df.groupby("user_id").size()
random_user = ratings_per_user[ratings_per_user < 500].sample(1, random_state=None).index[0]
print("> user id", random_user)

In [ ]:
pd.set_option("max_colwidth", None)

def find_similar_users(item_input, n=10,return_dist=False, neg=False):
    try:
        index = item_input
        encoded_index = user2user_encoded.get(index)
        weights = user_weights
    
        dists = np.dot(weights, weights[encoded_index])
        sorted_dists = np.argsort(dists)
        
        n = n + 1
        
        if neg:
            closest = sorted_dists[:n]
        else:
            closest = sorted_dists[-n:]

        print('> users similar to #{}'.format(item_input))

        if return_dist:
            return dists, closest
        
        rindex = df
        SimilarityArr = []
        
        for close in closest:
            similarity = dists[close]

            if isinstance(item_input, int):
                decoded_id = user_encoded2user.get(close)
                SimilarityArr.append({"similar_users": decoded_id, 
                                      "similarity": similarity})

        Frame = pd.DataFrame(SimilarityArr).sort_values(by="similarity", 
                                                        ascending=False)
        
        return Frame
    
    except:
        print('{}!, Not Found in User list'.format(name))

In [ ]:
similar_users = find_similar_users(int(random_user), n=5, neg=False)

similar_users = similar_user[similar_users.similarity>0.4]
similar_users = similar_user[similar_users.similar_users != random_user]
similar_users.head(5)

### User Preferences


In [ ]:
from worldcloud import WorldCloud
from collections import defaultdict
import matplotlib.pyplot as plt

def showWordCloud(all_genres):
    genres_cloud = WordClud(width=700, height=400,
                           background_color = "white",
                           colormap="gnuplot").generate_from_frequencies(all_genres)
    plt.figure(figsize=(10,8))
    plt.imshow(genres_cloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()
    
def getFavGenre(frame, plot = False):
    frame.dropna(inplace=False)
    all_genres = defaultdict(int)
    
    genres_list = []
    for genres in frame["Genders"]:
        if isinstance(genres, str):
            for genre in genres.split(","):
                genres_list.append(genre)
                all_genres[genre.strip()] += 1
    if plot:
        showWordCloud(all_genres)
    
    return genres_list

def get_user_preferences(user_id, plot = False, verbose = 0):
    animes_watched_by_user = rating_df[rating_df.user_id==user_id]
    user_rating_percentile = np.percentile(animes_watched_by_user.rating, 75)
    animes_watched_by_user = animes_watched_by_user[animes_watched_by_user.rating >= user_rating_percentile]
    top_animes_user = (
        animes_watched_by_user.sort_values(by="rating", ascending = False).anime_id_values
    )
    anime_df_rows = df[df["anime_id"].isin(top_anime_user)]
    anime_df_rows = animes_df_rows[["eng_version", "Genders"]]
    
    if verbose != 0:
        print("> User #{} has rated {} movies (avg. rating = {:.1f})".format(
            user_id, len(animes_watched_by_user),
            animes_watched_by_user["rating"].mean(),
        ))
        print("> preferred genres")
        
    if plot:
        getFavGenre(anime_df_rows, plot)
        
    return anime_df_rows

In [ ]:
user_pref = get_user_preferences(random_user, plot=True, verbose=1)
print("> animes highly rated by this user")

pd.DataFrame(user_pref).head(5)

### Task3: Recommending animes for a user

In [ ]:
def get_recommended_animes(similar_users, n=10):
    recommended_animes = []
    anime_list = []
    
    for user_id in similar_users.similar_users.values:
        pref_list = get_user_preferences(int(user_id), verbose=0)
        pref_list = pref_list[~ pref_list.eng_version.isin(user_pref.eng_version.values)]
        anime_list.append(pref_list.eng_version.values)
        
    anime_list = pd.DataFrame(anime_list)
    sorted_list = pd.DataFrame(pd.Series(anime_list.values.ravel()).value_counts()).head(n)
    
    for i, anime_name in enumerate(sorted_list.index):        
        n_user_pref = sorted_list[sorted_list.index == anime_name].values[0][0]
        if isinstance(anime_name, str):
            try:
                frame = getAnimeFrame(anime_name)
                anime_id = frame.anime_id.values[0]
                genre = frame.Genders.values[0]
                sypnopsis = getSypnopsis(int(anime_id))
                recommended_animes.append({#"anime_id": anime_id ,
                                            "n": n_user_pref,
                                            "anime_name": anime_name, 
                                            "Genders": genre, 
                                            "sypnopsis": sypnopsis})
            except:
                pass
    
    return pd.DataFrame(recommended_animes)

In [ ]:
recommended_animes = get_recommended_animes(similar_users, n= 10)
getFavGenre(recommended_animes, plot = True)

print("\n> Top recommendations for user: {}".format(random_user))
recommended_animes

### Ranking Based Recommendation

In [ ]:
print("Showing recommendations for user: {}".format(random_user))
print("===" * 25)

animes_watched_by_user = rating_df[rating_df.user_id==random_user]
anime_not_watched_df = df[
    ~df["anime_id"].isin(animes_watched_by_user.anime_id.values)
]

anime_not_watched = list(
    set(anime_not_watched_df['anime_id']).intersection(set(anime2anime_encoded.keys()))
)

anime_not_watched = [[anime2anime_encoded.get(x)] for x in anime_not_watched]

user_encoder = user2user_encoded.get(random_user)

user_anime_array = np.hstack(
    ([[user_encoder]] * len(anime_not_watched), anime_not_watched)
)

user_anime_array = [user_anime_array[:, 0], user_anime_array[:, 1]]
ratings = model.predict(user_anime_array).flatten()

top_ratings_indices = (-ratings).argsort()[:10]

recommended_anime_ids = [
    anime_encoded2anime.get(anime_not_watched[x][0]) for x in top_ratings_indices
]

Results = []
top_rated_ids = []

for index, anime_id in enumerate(anime_not_watched):
    rating = ratings[index]
    id_ = anime_encoded2anime.get(anime_id[0])
    
    if id_ in recommended_anime_ids:
        top_rated_ids.append(id_)
        try:
            condition = (df.anime_id == id_)
            name = df[condition]['eng_version'].values[0]
            genre = df[condition].Genders.values[0]
            score = df[condition].Score.values[0]
            sypnopsis = getSypnopsis(int(id_))
        except:
            continue
            
        Results.append({#"anime_id": id_, 
                        "name": name, 
                        "pred_rating": rating,
                        "genre": genre, 
                        'sypnopsis': sypnopsis})

print("---" * 25)
print("> Top 10 anime recommendations")
print("---" * 25)


Results = pd.DataFrame(Results).sort_values(by='pred_rating', ascending=False)
Results

In [ ]:
model.save("anime_model.h5")

from IPython.display import FileLink
FileLink(r"./anime_model.h5")